### Basic Tic-Tac-Toe Setup

In [1]:
WIN_PATTERNS = [
    (0, 1, 2), (3, 4, 5), (6, 7, 8), # rows
    (0, 3, 6), (1, 4, 7), (2, 5, 8), # columns
    (0, 4, 8), (2, 4, 6) # diagonals
]

def check_if_won(board):
    for pattern in WIN_PATTERNS:
        a,b,c = pattern
        if board[a] == board[b] == board[c] != 0:
            return board[a]
    return 0

def apply_move(board, move, player):
    new_board = board.copy()
    new_board[move] = player
    return new_board

def get_possible_moves(board):
    return [i for i, x in enumerate(board) if x == 0]

def print_board(board):
    for i in range(3):
        for j in range(3):
            c = str(board[i * 3 + j])
            if c == "1":
                print("X", end=" ")
            elif c == "-1":
                print("O", end=" ")
            else:
                print(i * 3 + j, end=" ")
        print()
    print()


### Minimax Algorithm 

In [2]:
states = {}

def minimax(board, depth, player):
    key = (tuple(board))
    if key in states:
        return states[key]

    winner = check_if_won(board)
    if winner == 1:
        states[key] = 10 - depth
        return 10 - depth
    if winner == -1:
        states[key] = depth - 10
        return depth - 10
    
    possible_moves = get_possible_moves(board)
    if not possible_moves:
        states[key] = 0
        return 0

    if player == 1: # maximizer
        best_val = float('-inf')
        for move in possible_moves:
            best_val = max(best_val, minimax(apply_move(board, move, 1), depth + 1, -1))
        states[key] = best_val
        return best_val
    else: # ,inimizer
        best_val = float('inf')
        for move in possible_moves:
            best_val = min(best_val, minimax(apply_move(board, move, -1), depth + 1, 1))
        states[key] = best_val
        return best_val

# this only generates the states where the maximizer starts, saving half the states
minimax([0] * 9, 0, 1)
print(len(list(states)))


5478


### Test out the Minimax Algorithm states
If the AI plays optimally, it can only tie or win. We will simulate games against an opponent that plays randomly

In [3]:
import random

def get_best_move(board, ai_player):
    best_value = -float('inf')
    best_moves = []
    possible_moves = get_possible_moves(board)
    for i in possible_moves:
        new_board = apply_move(board, i, ai_player)
        value = states[tuple(new_board)]

        if ai_player == 1: # maximizer
            if value > best_value:
                best_value = value
                best_moves = [i]
            elif value == best_value:
                best_moves.append(i)

    return random.choice(best_moves)

def play_simulated_game(player):
    board = [0] * 9
    current_player = player

    while True:
        winner = check_if_won(board)
        if winner != 0:
            return winner
        if all(x != 0 for x in board):
            return 0

        if current_player == player:
            move = None
            if player == 1:
                move = get_best_move(board, player)
            # earler we only generated states for maximizer starting first, we invert the board so we can use the same states
            else: 
                move = get_best_move([i * -1 for i in board], 1)
            board = apply_move(board, move, player)
        else:
            move = random.choice(get_possible_moves(board))
            board = apply_move(board, move, -1 * player)

        current_player *= -1

from collections import Counter

def simulate():
    ai = random.choice((1, -1))
    winner = play_simulated_game(ai)
    return "Tie" if winner == 0 else ("AI" if winner == ai else "Human")

counter = Counter(simulate() for _ in range(5000))
print(dict(counter))


{'AI': 4846, 'Tie': 154}


### Store the states in a C++ Header File for Arduino
The arduino has 32kb of flash memory so to store the states we can use the location of the state in an array as the board hash. This means only using a single byte per state to value mapping. 

There are 9 cells on the board with 3 possible values. This means we can multiply each cell value by 3^index and sum them to get a unique hash.

The highest possible hash is when all cells are 2 (-1 previously) which means the the largest hash is:

(3^0 + 3^1 + 3^2 + 3^3 ) + 2 * (3^4 + 3^5 +3^6 + 3^7 + 3^8) = 19642

This also means the total memory needed is 19682 bytes which fits in the arduino's flash memory.


In [4]:
def generate_arduino_header(minimax_data, filename="move_table.h"):
    # 3^9 possible combinations
    TOTAL_STATES = 19642
    
    # initialize the table with 255 (represents 'no move' or 'illegal state')
    table = [255] * TOTAL_STATES

    def get_hash(state):
        total = 0
        for i, cell in enumerate(state):
            if cell == -1:
                cell = 2
            total += cell * (3**i)
        return total

    for board, best_move in minimax_data.items():
        h = get_hash(board)
        table[h] = best_move

    # write to a c++ header file
    with open(filename, "w") as f:
        f.write("#ifndef MOVE_TABLE_H\n#define MOVE_TABLE_H\n\n")
        f.write("#include <avr/pgmspace.h>\n\n")
        f.write(f"// Total Memory Usage: {TOTAL_STATES} bytes\n")
        f.write("const uint8_t moveTable[] PROGMEM = {\n")
        
        for i in range(0, TOTAL_STATES, 12):
            chunk = table[i:i+12]
            line = ", ".join(f"{x:>3}" for x in chunk)
            f.write(f"    {line},   // {i}-{i+11}\n")
            
        f.write("};\n\n#endif")
    
    print(f"Success! {filename} created.")

generate_arduino_header(states)


Success! move_table.h created.


If you open up `move_table.h` you can see that there are actually very few states that are used (5478 / 19642 ~ 28%). You may think that we could use a hash map to store only the used state, but this would require storing the hash keys as well which would take up more space than just storing the full table.